#[Your_name]

In [1]:
# !pip install arabert
# !pip install camel-tools
# !camel_data -i light
# !pip install transformers
# !pip uninstall camel-tools
# !pip uninstall arabert

<h1 style="text-align: center;">Sequence Labeling<h1>

Prepare libraries

In [83]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
import re
import nltk
from nltk import pos_tag, word_tokenize
from nltk.chunk import ne_chunk
import pandas as pd
from tokenization import tokenize as tk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.tagger.default import DefaultTagger
from sklearn.linear_model import LogisticRegression
# from camel_tools.utils.dediac import dediac_arabic_text
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from nltk.tag import UnigramTagger, BigramTagger

Download data

In [3]:
# !wget 'https://drive.google.com/uc?export=download&id=1-3JRPU4aq9t_RRujrJ1HpmAPUbRe0QYK' -O 'IOB.csv'

<h2 dir="rtl">مثال عن كيفية كتابة حلول الطلبات:</h2>


<div dir="rtl">شرح ما يقوم به الكود (like code documentation)<div>

In [4]:
# your code here


In [5]:
# example test

<div dir="rtl">ملاحظاتك في حال وجودها</div>
<div dir="rtl">يمكنك إضافة خلايا لكل طلب بقدر ما تشاء، المهم أن تحافظ على تنظيم الملف</div>

# Question [1]: Data Preparation

## [1.1]

In [84]:
path = 'IOB.csv'

def read_file(path,file):
    schedule = pd.read_csv(path, names=['word','entity tag','POS','stopword'],sep=',',skiprows=1)
    schedule['file'] = file
    return schedule


schedual = read_file(path,'IOB')
schedual.head(5)

,word,entity tag,POS,stopword,file
0,إن,O,verb_pseudo,yes,IOB
1,اللوكيميا,B,noun,no,IOB
2,أو,O,conj,yes,IOB
3,(,O,punc,no,IOB
4,ابيضاض,B,noun,no,IOB


In [85]:
entity = set(schedual['entity tag'].values)
entity

{'B', 'I', 'O'}

In [87]:
def lines_to_tuples(words,tags):
   
    entities = []
    for word,tag in zip(words,tags):
        my_tuple = (word,tag)
        entities.append(my_tuple)

    return entities

# مثال على استخدام الدالة
result = lines_to_tuples(schedual['word'].values,schedual['entity tag'].values)
print(result)

[('إن', 'O'), ('اللوكيميا', 'B'), ('أو', 'O'), ('(', 'O'), ('ابيضاض', 'B'), ('الدم', 'I'), (')', 'O'), ('هو', 'O'), ('سرطان', 'B'), ('خلايا', 'I'), ('الدم', 'I'), ('البيضاء', 'I'), (',', 'O'), ('و', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('البيضاء', 'O'), ('تساعد', 'O'), ('الجسم', 'O'), ('على', 'O'), ('محاربة', 'O'), ('العدوى', 'O'), ('.', 'O'), ('تتشكل', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('في', 'O'), ('نقي', 'O'), ('العظام', 'O'), (',', 'O'), ('أما', 'O'), ('في', 'O'), ('اللوكيميا', 'B'), ('ف', 'O'), ('يقوم', 'O'), ('نقي', 'O'), ('العظام', 'O'), ('على', 'O'), ('كل', 'O'), ('حال', 'O'), ('ب', 'O'), ('إنتاج', 'O'), ('خلايا', 'O'), ('دم', 'O'), ('بيضاء', 'O'), ('غير', 'O'), ('طبيعية', 'O'), (',', 'O'), ('ب', 'O'), ('حيث', 'O'), ('تتجمع', 'O'), ('تلك', 'O'), ('الخلايا', 'O'), ('حول', 'O'), ('خلايا', 'O'), ('الدم', 'O'), ('الصحيحة', 'O'), ('مما', 'O'), ('يجعل', 'O'), ('من', 'O'), ('الصعب', 'O'), ('أن', 'O'), ('تقوم', 'O'), ('هذه', 'O'), ('الخلايا', 'O'), ('ب', 'O'), ('عملها', 'O'), ('ب', 

## [1.2]

In [89]:
phrases = []
def lines_to_phrases(words,tags,poss,stopwords):
    for word,tag,pos,stopword in zip(words,tags,poss,stopwords):
        phrase = ' '.join([word,tag,pos,stopword])
        phrases.append(phrase)


    return phrases

result = lines_to_phrases(schedual['word'].values,schedual['entity tag'].values,schedual['POS'].values,schedual['stopword'].values)
print(result)


['إن O verb_pseudo yes', 'اللوكيميا B noun no', 'أو O conj yes', '( O punc no', 'ابيضاض B noun no', 'الدم I noun no', ') O punc no', 'هو O pron yes', 'سرطان B noun no', 'خلايا I noun no', 'الدم I noun no', 'البيضاء I adj no', ', O punc no', 'و O conj no', 'خلايا O noun no', 'الدم O noun no', 'البيضاء O adj no', 'تساعد O verb no', 'الجسم O noun no', 'على O prep yes', 'محاربة O noun no', 'العدوى O noun no', '. O punc no', 'تتشكل O verb no', 'خلايا O noun no', 'الدم O noun no', 'في O prep yes', 'نقي O noun no', 'العظام O noun no', ', O punc no', 'أما O part_focus yes', 'في O prep yes', 'اللوكيميا B noun no', 'ف O conj yes', 'يقوم O verb no', 'نقي O noun no', 'العظام O noun no', 'على O prep yes', 'كل O noun_quant yes', 'حال O noun no', 'ب O prep no', 'إنتاج O noun no', 'خلايا O noun no', 'دم O noun no', 'بيضاء O adj no', 'غير O noun yes', 'طبيعية O adj no', ', O punc no', 'ب O prep no', 'حيث O adv_rel yes', 'تتجمع O verb no', 'تلك O pron_dem yes', 'الخلايا O noun no', 'حول O noun no', 'خلا

## [1.3]

In [90]:
list_of_lists = []
def lines_to_list_of_lists(words,tags,poss,stopwords):
    my_tuples  = lines_to_tuples(words,tags)
    my_phrases = lines_to_phrases(words,tags,poss,stopwords)

    for my_tuple,my_phrase in zip(my_tuples,my_phrases):
        list_of_lists.append([my_tuple,my_phrase])

    return list_of_lists    


lines_to_list_of_lists(schedual['word'].values,schedual['entity tag'].values,schedual['POS'].values,schedual['stopword'].values)

[[('إن', 'O'), 'إن O verb_pseudo yes'],
 [('اللوكيميا', 'B'), 'اللوكيميا B noun no'],
 [('أو', 'O'), 'أو O conj yes'],
 [('(', 'O'), '( O punc no'],
 [('ابيضاض', 'B'), 'ابيضاض B noun no'],
 [('الدم', 'I'), 'الدم I noun no'],
 [(')', 'O'), ') O punc no'],
 [('هو', 'O'), 'هو O pron yes'],
 [('سرطان', 'B'), 'سرطان B noun no'],
 [('خلايا', 'I'), 'خلايا I noun no'],
 [('الدم', 'I'), 'الدم I noun no'],
 [('البيضاء', 'I'), 'البيضاء I adj no'],
 [(',', 'O'), ', O punc no'],
 [('و', 'O'), 'و O conj no'],
 [('خلايا', 'O'), 'خلايا O noun no'],
 [('الدم', 'O'), 'الدم O noun no'],
 [('البيضاء', 'O'), 'البيضاء O adj no'],
 [('تساعد', 'O'), 'تساعد O verb no'],
 [('الجسم', 'O'), 'الجسم O noun no'],
 [('على', 'O'), 'على O prep yes'],
 [('محاربة', 'O'), 'محاربة O noun no'],
 [('العدوى', 'O'), 'العدوى O noun no'],
 [('.', 'O'), '. O punc no'],
 [('تتشكل', 'O'), 'تتشكل O verb no'],
 [('خلايا', 'O'), 'خلايا O noun no'],
 [('الدم', 'O'), 'الدم O noun no'],
 [('في', 'O'), 'في O prep yes'],
 [('نقي', 'O'), 'ن

## [1.4]

In [91]:
words_list = []
tags_list = []

def two_lists_word_and_tag(words,tags):
    for word,tag in zip(words,tags):
        words_list.append(word)
        tags_list.append(tag)

    return words_list,tags_list 


my_word_list,my_tag_list = two_lists_word_and_tag(schedual['word'].values,schedual['entity tag'].values)
print(my_word_list,my_tag_list)

['إن', 'اللوكيميا', 'أو', '(', 'ابيضاض', 'الدم', ')', 'هو', 'سرطان', 'خلايا', 'الدم', 'البيضاء', ',', 'و', 'خلايا', 'الدم', 'البيضاء', 'تساعد', 'الجسم', 'على', 'محاربة', 'العدوى', '.', 'تتشكل', 'خلايا', 'الدم', 'في', 'نقي', 'العظام', ',', 'أما', 'في', 'اللوكيميا', 'ف', 'يقوم', 'نقي', 'العظام', 'على', 'كل', 'حال', 'ب', 'إنتاج', 'خلايا', 'دم', 'بيضاء', 'غير', 'طبيعية', ',', 'ب', 'حيث', 'تتجمع', 'تلك', 'الخلايا', 'حول', 'خلايا', 'الدم', 'الصحيحة', 'مما', 'يجعل', 'من', 'الصعب', 'أن', 'تقوم', 'هذه', 'الخلايا', 'ب', 'عملها', 'ب', 'شكل', 'عادي', '.', 'هناك', 'أنواع', 'مختلفة', 'من', 'اللوكيميا', ',', 'و', 'تتضمن', 'هذه', 'الأنواع', ':', 'ابيضاض', 'اللمفاويات', 'الحاد', 'الابيضاض', 'النخاعي', 'الحاد', 'ابيضاض', 'اللمفاويات', 'المزمن', 'الابيضاض', 'النخاعي', 'المزمن', 'يمكن', 'ل', 'اللوكيميا', 'أن', 'تتطور', 'ب', 'سرعة', 'أو', 'ب', 'بطء', ',', 'ف', 'اللوكيميا', 'المزمنة', 'تتطور', 'ب', 'بطء', ',', 'أما', 'في', 'اللوكيميا', 'الحادة', 'فتكون', 'الخلايا', 'شاذة', 'جدا', 'عن', 'الوضع', 'الطبيعي', '

## [1.5]

In [92]:
from sklearn.model_selection import train_test_split

X = schedual[['word', 'POS', 'stopword']]
y = schedual['entity tag']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1000)



## [1.6]

In [93]:
# عدد الجمل في كل من الترين والتيست

train_data_phrases_len = len(X_train['word'])
test_data_phrases_len = len(X_test['word'])
print(f'Train phrases length: {train_data_phrases_len}')
print(f'Test phrases length: {test_data_phrases_len}')

Train phrases length: 50004
Test phrases length: 12502


## [1.7]

In [94]:
def calculate_named_entities_ratio(x_train_data,y_train_data):
    words_len = len(x_train_data['word'])
    tags_len = 0
    for i in y_train_data:
        if i == 'I' or i == 'B':
            tags_len +=1
    
    return tags_len/words_len
    
    
    
ratio = calculate_named_entities_ratio(X_train,y_train)
print(ratio)

0.04515638748900088


## [1.8]

In [95]:
from farasa.pos import FarasaPOSTagger
from farasa.segmenter import FarasaSegmenter

arabic_text = "Tony botros is the best in the whole word"

# Initialize the Farasa Segmenter in Standalone mode
farasa_segmenter = FarasaSegmenter(interactive=False)

# Tokenize the Arabic text
arabic_words = farasa_segmenter.segment(arabic_text)

# Initialize the Farasa POS tagger in Standalone mode
farasa_pos_tagger = FarasaPOSTagger(interactive=False)

# Perform part-of-speech tagging
pos_tags = farasa_pos_tagger.tag(arabic_words)

print(pos_tags)


S/S Tony/FOREIGN botros/FOREIGN is/FOREIGN the/FOREIGN best/FOREIGN in/FOREIGN the/FOREIGN whole/FOREIGN word/FOREIGN E/E


## [1.9]

In [96]:
count_per_category = y_train.value_counts()

most_common_category = count_per_category.idxmax()
print(f"الصنف الأكثر تكرارًا هو: {most_common_category}")


الصنف الأكثر تكرارًا هو: O


# Question [2]: Statistics

In [97]:
sentence = "التصلب المتعدد مرض يحتمل أن يسبب إعاقة الدماغ و الحبل النخاعي (الجهاز العصبي المركزي). عند الإصابة بمرض التصلب المتعدد، يهاجم الجهاز المناعي غمد الحماية (المايلين) الذي يغطي الألياف العصبية، ويسبب مشكلات في الاتصال بين الدماغ وبقية الجسم. في النهاية، يمكن أن يسبب تلفاً أو تدهوراً دائما في الألياف العصبية.  "

In [98]:
sentence_tokenize=tk(sentence)

In [99]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['accuracy'] = []

## [2.1]


In [127]:
print(len(schedual[schedual['entity tag']=='O']))

59674


In [123]:
print(len(schedual))

62506


In [116]:
def taget_labet_classifier(words):
    return [(word,'O') for word in words]

In [118]:
classified_text = taget_labet_classifier(sentence_tokenize)
# classified_text

In [131]:
def calculate_accuracy(predicted_labels, true_labels):
    correct = sum(pred == true for pred, true in zip(predicted_labels, true_labels))
    accuracy = correct / len(predicted_labels) if predicted_labels else 0
    return accuracy

In [133]:
predicted_labels = taget_labet_classifier(schedual['word'])

true_labels = schedual['entity tag'] 

true_labels_pairs = [(word, tag) for word, tag in zip(schedual['word'], true_labels)]

accuracy = calculate_accuracy(predicted_labels, true_labels_pairs)
print(f"accuracy : {accuracy}")

دقة الدالة: 0.9546923495344447


## [2.2]

In [100]:
from nltk import FreqDist

# word_tokens = word_tokenize(' '.join(X_train['word'].values))
word_tokens = tk(' '.join(X_train['word'].values))
freq_dist = FreqDist(word_tokens)
top_100_words = freq_dist.most_common(100) 
# print(top_100_words)
top_100_words_only = [word for word, _ in top_100_words]
# print(top_100_words_only)


In [101]:
filtered_schedual = schedual[schedual['word'].isin(top_100_words_only)].drop_duplicates().dropna()

In [102]:
tagger_lookup = dict(zip(filtered_schedual['word'], filtered_schedual['entity tag']))

def classify_words(words):
    return [(word, tagger_lookup.get(word,'')) for word in words]

In [103]:
classified_text = classify_words(sentence_tokenize)
# classified_text

In [134]:
predicted_labels = classify_words(schedual['word'])

true_labels = schedual['entity tag'] 

true_labels_pairs = [(word, tag) for word, tag in zip(schedual['word'], true_labels)]

accuracy = calculate_accuracy(predicted_labels, true_labels_pairs)
print(f"accuracy : {accuracy}")

دقة الدالة: 0.9546923495344447


In [104]:
default_tag = 'O'
def classify_with_backoff(words):
    return [(word, tagger_lookup.get(word, default_tag)) for word in words]

In [105]:
classified_text = classify_with_backoff(sentence_tokenize)
# classified_text

In [135]:
predicted_labels = classify_with_backoff(schedual['word'])

true_labels = schedual['entity tag'] 

true_labels_pairs = [(word, tag) for word, tag in zip(schedual['word'], true_labels)]

accuracy = calculate_accuracy(predicted_labels, true_labels_pairs)
print(f"accuracy : {accuracy}")

دقة الدالة: 0.851182286500496


## [2.3]

In [106]:
X_train, X_test, y_train, y_test = train_test_split(schedual['word'], schedual['entity tag'], test_size=0.2, random_state=1000)
train_data=dict(zip(X_train, y_train))
test_data=dict(zip(X_test,y_test))
tagger_list = [(word, tag) for word, tag in train_data.items()]
train_data = [[pair] for pair in tagger_list]
tagger_list = [(word, tag) for word, tag in test_data.items()]
test_data = [[pair] for pair in tagger_list]

In [137]:
unigram_tagger = UnigramTagger(train_data)

unigram_accuracy = unigram_tagger.accuracy(test_data)
print(f"unigram_accuracy : {unigram_accuracy}")

unigram_accuracy : 0.781634446397188


In [138]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)

bigram_accuracy = bigram_tagger.accuracy(test_data)
print(f"bigram_accuracy : {bigram_accuracy}")

bigram_accuracy : 0.781634446397188


# Question [3]: ML Classification

## [3.1]


### [3.1.1]

### [3.1.2]

### [3.1.3]

### [3.1.4]

### [3.1.5]

### [3.1.6]

### [3.1.7]

## [3.2]

## [3.3]

## [3.4]

## [3.5]

# Question [4]: DL Classification

## [4.1]


## [4.2]

## [4.3]

The Comparison Dictionary example

In [ ]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['accuracy'] = []

In [ ]:
import pandas as pd

df = pd.DataFrame(comparison_table)
df

In [ ]:
df.to_csv("[your_name].csv", index=False)

<h1 style="text-align: center;">Sequence to Sequence<h1>

Prepare libraries

In [ ]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
!pip install arabert

Download data

In [ ]:
# !wget 'https://drive.google.com/uc?export=download&id=1OoUGXNKnmm3KYabreBFWMV9mr2JwQOJH' -O 'test.csv'
# !wget 'https://drive.google.com/uc?export=download&id=1xnw6kKitwQwYgaOAQppDbj6T3ARCwRqz' -O 'train.csv'

In [ ]:
from arabert import ArabertPreprocessor
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel

# [Extra 1]

# [2]

# [3]